E

# Importing the libraries

In [ ]:
# reading in data
import re
import string
import numpy as np 
import random
import pandas as pd 

#Vissualizations
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

#Preprocessing
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

#modeling
from sklearn.feature_extraction.text import CountVectorizer

#csv paths
import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Below is a helper Function which generates random colors which will be used to give different colors to our plots.

In [ ]:
def random_colours(number_of_colors):
    '''
    Simple function for random colours generation.
    Input:
        number_of_colors - integer value indicating the number of colours which are going to be generated.
    Output:
        Color in the following format: ['#E86DA4'] .
    '''
    colors = []
    for i in range(number_of_colors):
        colors.append("#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]))
    return colors

# Getting Data

In [ ]:
# Load dataset
train_df = pd.read_csv('/kaggle/input/climate-change-belief-analysis/train.csv')
test_df = pd.read_csv('/kaggle/input/climate-change-belief-analysis/test.csv')
ss_df = pd.read_csv('/kaggle/input/climate-change-belief-analysis/sample_submission.csv')

In [ ]:
train_df.head()

In [ ]:
print('___basic info of the train data___')
print(train_df.info())
print('Dataset size:', train_df.shape)
print('Columns are:',train_df.columns)

print('___Print the head/Tain of the train data_____')
print(train_df.head())
print('________________________')
print(train_df.tail())

As seen above, when we print some basic information about the data. We can notice right away how each row is represented either by int64 or Object (for TextData). Finally after plotting the histogram of the sentiment data, its clear that there more positive examples then negative examples. (1 being denoted as positive sentiment while 0 being denoted as negative.). There are four sentiments in total.

# Data cleaning and Preprocessing

Replacing contaction phrases 

In [ ]:
contractions_dict = contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}


def expand_contractions(text):
    for word in text.split():
        if word.lower() in contractions_dict:
            text = text.replace(word,contractions_dict[word.lower()])
    return text

train_df['fixed_cntractions'] = train_df['message'].apply(lambda x: expand_contractions(x))
test_df['fixed_cntractions'] = test_df['message'].apply(lambda x: expand_contractions(x))

### Dealig with the punctuations and the links present in the messages

In [ ]:
def remov_pnct(text):
    '''The Function makes text lowercase,removes links, punctuation(!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~)
    and words containing numbers.'''
    
    text = str(text).lower() #Make text lowercase
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())# remove @user, #word and link
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) ##remove punctuation
    text = re.sub('\n', '', text)
    text = ''.join([i for i in text[:] if not i.isdigit()]) #remove numbers
    return text

train_df['clean_message'] = train_df['fixed_cntractions'].apply(lambda x: remov_pnct(x))
test_df['clean_message'] = test_df['fixed_cntractions'].apply(lambda x: remov_pnct(x))

In [ ]:
train_df.head()

# Creating features

Getting the number of clean words and comparing them with that  raw from the message and clean message columns.

In [ ]:
#train data frame
train_df['Num_words_raw'] = train_df['message'].apply(lambda x:len(str(x).split())) 
train_df['Num_words_clean'] = train_df['clean_message'].apply(lambda x:len(str(x).split()))
#Difference in Number of words text and Selected Text
train_df['difference_in_wordsNo'] = abs(train_df['Num_words_raw'] - train_df['Num_words_clean']) 
#test data frame
test_df['Num_words_raw'] = test_df['message'].apply(lambda x:len(str(x).split())) 
test_df['Num_words_clean'] = test_df['clean_message'].apply(lambda x:len(str(x).split())) 
#Difference in Number of words text and Selected Text
test_df['difference_in_wordsNo'] = abs(test_df['Num_words_raw'] - test_df['Num_words_clean'])

In [ ]:
train_df.head()  

# Exploratory Data Analysis

Taking a closer look at the distribution of tweeter messages in the train set.

In [ ]:
temp = train_df.groupby('sentiment').count()['message'].reset_index().sort_values(by='message',ascending=False)
temp.style.background_gradient(cmap='Purples')

In [ ]:
fig = px.funnel(temp, x='sentiment', y='message', color='sentiment')
fig.show()

### Wordcloud Visualization

In [ ]:
pd.set_option('display.max_colwidth', 100)
# visualising the messages 
df_senti1 = train_df[train_df['sentiment']==1]
df_senti0 = train_df[train_df['sentiment']==0]
df_senti_neg1 = train_df[train_df['sentiment']==-1]
df_senti2 = train_df[train_df['sentiment']==2]
tweet_All = " ".join(review for review in train_df.message)
tweet_senti1 = " ".join(review for review in df_senti1.message)
tweet_senti0 = " ".join(review for review in df_senti0.message)
tweet_senti_neg1 = " ".join(review for review in df_senti_neg1.message)
tweet_senti2 = " ".join(review for review in df_senti2.message)

fig, ax = plt.subplots(5, 1, figsize  = (65,65))
# Create and generate a word cloud image:
wordcloud_ALL = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_All)
wordcloud_1 = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_senti1)
wordcloud_0 = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_senti0)
wordcloud_neg1 = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_senti_neg1)
wordcloud_2 = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_senti2)

# Display the generated image:
ax[0].imshow(wordcloud_ALL, interpolation='bilinear')
ax[0].set_title('All Tweets', fontsize=50)
ax[0].axis('off')
ax[1].imshow(wordcloud_1, interpolation='bilinear')
ax[1].set_title('Tweets under Pro Class 1',fontsize=50)
ax[1].axis('off')
ax[2].imshow(wordcloud_0, interpolation='bilinear')
ax[2].set_title('Tweets under Neutral Class 0',fontsize=50)
ax[2].axis('off')
ax[3].imshow(wordcloud_neg1, interpolation='bilinear')
ax[3].set_title('Tweets under Anti Class -1',fontsize=50)
ax[3].axis('off')
ax[4].imshow(wordcloud_2, interpolation='bilinear')
ax[4].set_title('Tweets under News Class 2',fontsize=50)
ax[4].axis('off')

### What is currently Know About the Data:

looking at some things that are already known about the data will helps in gaining more new insights:

* Clean_message is a cleaned version of message without links,#tags contraction words, punctuations, numericals, twitter handels and links
* Accorning to this discussion:https://www.kaggle.com/c/tweet-sentiment-extraction/discussion/138520 its now known that neutral tweets have a jaccard similarity of 97 percent between message and clean message.
* jaccard similarity - compares members for two sets to see which members are shared and which are distinct. It's a measure of similarity for the two sets of data, with a range from 0% to 100% ref(https://www.geeksforgeeks.org/find-the-jaccard-index-and-jaccard-distance-between-the-two-given-sets/)



In [ ]:
def jaccard(str1, str2): 
    '''It takes two strings can be two columns fro a df and returns intersection of twe two  divided by thier union'''
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
# on the train_df
results_jaccard=[]

for ind,row in train_df.iterrows():
    sentence1 = row.message
    sentence2 = row.clean_message

    jaccard_score = jaccard(sentence1,sentence2)
    results_jaccard.append([sentence1,sentence2,jaccard_score])

In [ ]:
jaccard = pd.DataFrame(results_jaccard,columns=["message","clean_message","jaccard_score"])
train_df = train_df.merge(jaccard,how='outer')

In [ ]:
train_df.head()

Diribution of number of words.

In [ ]:
hist_data = [train_df['Num_words_raw'], train_df['Num_words_clean']]

group_labels = ['message','clean_message']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels,show_curve=False)
fig.update_layout(title_text='Distribution of Number Of words')
fig.update_layout(
    autosize=False,
    width=900,
    height=700,
    paper_bgcolor="LightSteelBlue")
fig.show()

The number of words plot is really interesting ,the tweets messages having number of words greater than 18 are very less and thus the number of words distribution plot is a little bit skewed the right

In [ ]:
plt.figure(figsize=(12,6))
p1=sns.kdeplot(train_df['Num_words_raw'], shade=True, color="r").set_title('Distribution of Number Of words')
p2=sns.kdeplot(train_df['Num_words_clean'], shade=True, color="b")

Accordding to the above diagram, total numer of words in the message has been reduced.Hence the frequecy of message with less count of words incresed.

#### Checking the differnce in number of words and jaccard_scores across different Sentiments

In [ ]:
plt.figure(figsize=(12,6))
p1=sns.kdeplot(train_df[train_df['sentiment']== 1]['difference_in_wordsNo'], shade=True, color="b").set_title('Kernel Distribution of Difference in Number Of words')
p2=sns.kdeplot(train_df[train_df['sentiment']== -1]['difference_in_wordsNo'], shade=True, color="r")
p3=sns.kdeplot(train_df[train_df['sentiment']== 0]['difference_in_wordsNo'], shade=True, color="g")
p4=sns.kdeplot(train_df[train_df['sentiment']== 2]['difference_in_wordsNo'], shade=True, color="y")
plt.legend(labels=[1, -1, 0, 2])

sentiment -1(red) has the highest zero diffrence of words as compared to the other three and 1 having the lowest(blue).

In [ ]:
plt.figure(figsize=(12,6))
p1=sns.kdeplot(train_df[train_df['sentiment']==1]['jaccard_score'], cumulative=True, bw=2.5,shade=True, color="b").set_title('KDE of Jaccard Scores across different Sentiments')
p2=sns.kdeplot(train_df[train_df['sentiment']== -1]['jaccard_score'], cumulative=True, bw=2.5, shade=True, color="r")
p3=sns.kdeplot(train_df[train_df['sentiment']== 0]['jaccard_score'], cumulative=True, bw=2.5,shade=True, color="g")
p4=sns.kdeplot(train_df[train_df['sentiment']== 2]['jaccard_score'], cumulative=True, bw=2.5, shade=True, color="y")
plt.legend(labels=[1, -1, 0, 2])

### Most Common words in our clean_message 

In [ ]:
train_df['temp_list'] = train_df['clean_message'].apply(lambda x:str(x).split())
top = Counter([item for sublist in train_df['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Selected Text', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

While cleaning dataset we didnt remove the stop words were not removed and hence, it showing that most common word is she and is.


Now  checking coomon words after removing the stop words.

In [ ]:
stop = stopwords.words('english')
train_df['temp_list'] = train_df['temp_list'].apply(lambda x: [item for item in x if item not in stop])

In [ ]:
train_df.head()

In [ ]:
top = Counter([item for sublist in train_df['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Purples')

In [ ]:
fig = px.treemap(temp, path=['Common_words'], values='count',title='Tree of Most Common Words')
fig.show()

with stop words removed, now the most common word is climate 

#### Most Common words in raw messages
looking at the most common words in raw messages

In [ ]:
train_df['temp_list1'] = train_df['message'].apply(lambda x:str(x).split()) #List of words in every row for text 
stop = stopwords.words('english')
train_df['temp_list'] = train_df['temp_list1'].apply(lambda x: [item for item in x if item not in stop])#Removing Stopwords

In [ ]:
#data frame for top 20 most common words in message
top = Counter([item for sublist in train_df['temp_list1'] for item in sublist])
temp = pd.DataFrame(top.most_common(25))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Text', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

## Most common words sentiment wise

In [ ]:
Pro = train_df[train_df['sentiment']==1]
Anti = train_df[train_df['sentiment']== -1]
Neutral = train_df[train_df['sentiment']==0]
News = train_df[train_df['sentiment']== 2]

In [ ]:
#MosT common positive words(Pro)
top = Counter([item for sublist in Pro['temp_list'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(20))
temp_positive.columns = ['Common_words','count']
temp_positive.style.background_gradient(cmap='Greens')

In [ ]:
#MosT common negative words(Anti)
top = Counter([item for sublist in Anti['temp_list'] for item in sublist])
temp_negative = pd.DataFrame(top.most_common(20))
temp_negative = temp_negative.iloc[1:,:]
temp_negative.columns = ['Common_words','count']
temp_negative.style.background_gradient(cmap='Reds')

In [ ]:
#MosT common words in Neutral sentiment
top = Counter([item for sublist in Neutral['temp_list'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']
temp_neutral.style.background_gradient(cmap='Reds')

In [ ]:
#MosT common words in News sentiment
top = Counter([item for sublist in News['temp_list'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']
temp_neutral.style.background_gradient(cmap='Reds')

* We can see words like change,cllimate,global are common in all four segments. There are more positive occuring words than the negative.

* next is checking the word unique to different sentiments

Few high frequency tokens such as 'https', 'CO', 'RT' 'Climate change'are frequently used in all the categorical classes.

### Look at Unique Words in each Segment


unique words in each segment in the Following Order:

* Positive Pro (1)
* Negative Anti (-1)
* Neutral Neu (0)
* News        (2)

In [ ]:
raw_text = [word for word_list in train_df['temp_list1'] for word in word_list]

# def words_unique(sentiment,numwords,raw_words):
    '''
    Input:
        sentiment - sentiment category (ex. 'Neutral');
        numwords - how many specific words do you want to see in the final result; 
        raw_words - list  for item in train_ddf[train_df.sentiment == sentiment]['temp_list1']:
    Output: 
        dataframe giving information about the name of the specific ingredient and how many times it occurs in the chosen cuisine (in descending order based on their counts)..

    '''
    allother = []
    for item in train_df[train_df.sentiment != sentiment]['temp_list1']:
        for word in item:
            allother .append(word)
    allother  = list(set(allother ))
    
    specificnonly = [x for x in raw_text if x not in allother]
    
    mycounter = Counter()
    
    for item in train_df[train_df.sentiment == sentiment]['temp_list1']:
        for word in item:
            mycounter[word] += 1
    keep = list(specificnonly)
    
    for word in list(mycounter):
        if word not in keep:
            del mycounter[word]
    
    Unique_words = pd.DataFrame(mycounter.most_common(numwords), columns = ['words','count'])
    
    return Unique_words

### Positive message

#Unique_Positive= words_unique(1, 20, raw_text)
print("The top 20 unique words in Positive Tweets are:")
#Unique_Positive.style.background_gradient(cmap='Greens')

Unique_Positive= words_unique('0', 20, raw_text)
print("The top 20 unique words in Positive Tweets are:")
Unique_Positive.style.background_gradient(cmap='Greens')

C

### Tokenization


In [ ]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

train_df['tokenized_message'] = train_df['message_punct'].apply(lambda x: tokenization(x.lower()))
test_df['tokenized_message'] = test_df['message_punct'].apply(lambda x: tokenization(x.lower()))
train_df.head()

the tweet messages were coverted into list of word and to lower case

### Stemming and Lammitization

In [ ]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

train_df['message_stemmed'] = train_df['message_nonstop'].apply(lambda x: stemming(x))

test_df['message_stemmed'] = test_df['message_nonstop'].apply(lambda x: stemming(x))
train_df.head()

In [ ]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

train_df['message_lemmatized'] = train_df['message_nonstop'].apply(lambda x: lemmatizer(x))

test_df['message_lemmatized'] = test_df['message_nonstop'].apply(lambda x: lemmatizer(x))
train_df.head()

In [ ]:
ps = PorterStemmer()
train_df['message_stemmed'] = train_df['message_nonstop'].apply(lambda x: ' '.join([ps.stem(word) for word in x]))
test_df['message_stemmed'] = test_df['messsage_nonstop'].apply(lambda x: ' '.join([ps.stem(word) for word in x]))
train_df.head()

### Replace abbreviations and some spell correction

In [ ]:
#print('\n_________________________________')
#import csv,re

#def translator(user_string):
   # print(train_df['sentimentText1'])
    #train_df['sentimentText_abrv'] = train_df['sentimentText1'].apply(lambda x: translator(x))
    #print(train_df['sentimentText_abrv'])

### Stemming

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
print('_____________Stemmin________')
train_df['message_stemmed'] = train_df['messsage_nonstop'].apply(lambda x: ' '.join([ps.stem(word) for word in x]))
print(train_df['message_stemmed'])

So as seen above, the word evidence have changed to evid, as well as anthropogenic to anthropoge and many more others

In [ ]:
### Lemmatization

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
print(train_df['sentiment_stem'])
print('_________Lemmetazation______')
train_df['sentiment_lemma'] = train_df['sentiment_stem'].apply(lambda x: ' '.join([WordNetLemmatizer().lemmatize(word, 'v') for word in x.split()])) 
print(train_df['sentiment_lemma'])

#comment

### Parts of Speech Tagging (POS) 

In [ ]:
import nltk
print(train_df['sentiment_lemma'])
print('___________Parts of Speech Tagging (POS)_______________')
train_df['sentiment_POS'] = train_df['sentiment_lemma'].apply(lambda x: nltk.pos_tag(nltk.word_tokenize(x)))
print(train_df['sentiment_POS'])

### below is modeling of the raw data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = train_df['message']
y = train_df['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20,random_state = 42 )

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
count_vect = CountVectorizer()

In [ ]:
#fiit the vactorize to data build vocab 
#count_vect.fit(X_train)
#X_train_count = count_vect.transform(X_train)
#transform
X_train_count = count_vect.fit_transform(X_train)

In [ ]:
X_train_count

In [ ]:
X_train.shape

In [ ]:
X_train_count.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf_transformer = TfidfTransformer()

In [ ]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_count)

In [ ]:
X_train_tfidf.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
clf = LinearSVC()

In [ ]:
clf.fit(X_train_tfidf, y_train)

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [ ]:
text_clf.fit(X_train, y_train)

In [ ]:
predictions = text_clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
print(confusion_matrix(y_test,predictions))

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
from sklearn import metrics

In [ ]:
metrics.accuracy_score(y_test, predictions)

In [ ]:
metrics.f1_score(y_test, predictions, average='micro')

In [ ]:
X_train1 = train_df['message']
y_train1 = train_df['sentiment']
X_test1 = test_df['message']

In [ ]:
text_clf.fit(X_train1, y_train1)

In [ ]:
text_clf_pred = text_clf.predict(X_test1)

In [ ]:
text_clf_pred = pd.DataFrame(data=text_clf_pred,
                                 columns=['sentiment'],
                                 index=test_df['tweetid'])
text_clf_pred.reset_index(inplace=True)

In [ ]:
text_clf_pred

In [ ]:
text_clf_pred.to_csv("../output", index=False)

In [ ]:
text_clf_pred

checking

In [ ]:
train_df.head()

In [ ]:
#checking for blanks
train_df[train_df['message'] == ' '].index

the are no blanks in the massage entries

checking for duplicated entries

In [ ]:
duplicateRowsDF = train_df[train_df.duplicated(['message'])]

In [ ]:
duplicateRowsDF

the duplicated massages are the retweets.

In [ ]:
train_df.shape

In [ ]:
data = train_df.drop_duplicates(subset ="message", 
                     keep = False, inplace = True)

In [ ]:
dat = train_df.drop_duplicates(subset='message', keep='first')

In [ ]:
dat.shape

In [ ]:
dat

In [ ]:
dat_duplicated = dat[dat.duplicated(['message'])]

In [ ]:
dat_duplicated